# Designing and Creating a Database

## Index

* [Intro](#Intro)
* [Imports libraries and read data](#Imports-libraries-and-read-data)
  * [Helpers](#Helpers)
* [Getting to Know the Data](#Getting-to-Know-the-Data)
  * [game_log](#game_log)
  * [park_codes](#park_codes)
  * [person_codes](#person_codes)
  * [team_codes](#team_codes)
* [Additional info](#Additional-info)
  * [Position of defensive players](#Position-of-defensive-players)
  * [Leagues names and abbreviations](#Leagues-names-and-abbreviations)
  * [Offensive statistics meaning](#Offensive-statistics-meaning)
  * [Pitching statistics meaning](#Pitching-statistics-meaning)
  * [Defensive statistics](#Defensive-statistics)
* [Importing Data into SQLite](#Importing-Data-into-SQLite)
* [Planning a Normalized Schema](#Planning-a-Normalized-Schema)
* [Creating Tables](#Creating-Tables-Without-Foreign-Key-Relations)
* [Drop old tables](#Drop-old-tables)

---

## <a id='Intro'></a> Intro
[Top](#Index)

We will be working with a file of [Major League Baseball](https://en.wikipedia.org/wiki/Major_League_Baseball) games from [Retrosheet](http://www.retrosheet.org/). Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. The main file we will be working from `game_log.csv`, has been produced by combining 127 separate CSV files from retrosheet, and has been pre-cleaned to remove some inconsistencies. The game log has hundreds of data points on each game which we will normalize into several separate tables using SQL, providing a robust database of game-level statistics.

In addition to the main file, we have also included three 'helper' files, also sourced from Retrosheet:

* `park_codes.csv`
* `person_codes.csv`
* `team_codes.csv`

These three helper files in some cases contain extra data, but will also make things easier as they will form the basis for three of our normalized tables.

---

## Imports libraries and read data
[Top](#Index)

In [1]:
import pandas as pd
import sqlite3

In [2]:
data = {}

data_files = [
    "game_log.csv",
    "park_codes.csv",
    "person_codes.csv",
    "team_codes.csv",
    "appearance_type.csv"
]
data = {}

for file in data_files:
    key = file.replace('.csv', '')
    data[key] = pd.read_csv(file, low_memory=False)
    # delete leading and trailing whitespaces (including newlines) from column's names
    data[key].rename(str.strip, axis='columns', inplace=True)
    
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

### <a id='Helpers'></a> Helpers
[Top](#Index)

In [3]:
db_name = 'mlb.db'

def import_table(df, table_name):
    with sqlite3.connect(db_name) as conn:
        df.to_sql(table_name, conn, index=False, if_exists='replace')

def run_query(q):
    with sqlite3.connect(db_name) as conn:
        return pd.read_sql(q, conn)
    
def create_function(name, func):
    with sqlite3.connect(db_name) as conn:
        conn.create_function(name, len(signature(run_query).parameters), func)
    
def run_command(c):
    with sqlite3.connect(db_name) as conn:
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    return run_query('''
        SELECT
            name,
            type
        FROM sqlite_master
        WHERE type IN ("table","view");
    ''')

---

## <a id="Getting to Know the Data"></a>Getting to Know the Data
[Top](#Index)

In [4]:
for key in data:
    print(key, '\t', 'rows:', data[key].shape[0], '\t', 'cols:', data[key].shape[1])

game_log 	 rows: 171907 	 cols: 161
park_codes 	 rows: 252 	 cols: 9
person_codes 	 rows: 20494 	 cols: 7
team_codes 	 rows: 150 	 cols: 8
appearance_type 	 rows: 31 	 cols: 3


### <a id='game_log'></a> game_log
[Top](#Index)

In [5]:
game_log = data['game_log']
game_log.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [6]:
game_log.tail()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,4,60.0,D,NaN,NaN,NaN,DEN02,27762.0,203.0,0200000202,1100100010,39.0,10.0,4.0,1.0,2.0,6.0,0.0,0.0,1.0,4.0,0.0,12.0,2.0,1.0,0.0,0.0,8.0,7.0,4.0,4.0,1.0,0.0,30.0,12.0,1.0,0.0,0.0,0.0,41.0,13.0,4.0,0.0,1.0,4.0,1.0,0.0,1.0,3.0,0.0,11.0,0.0,1.0,0.0,0.0,12.0,5.0,6.0,6.0,0.0,0.0,30.0,13.0,0.0,0.0,0.0,0.0,barrs901,Scott Barry,woodt901,Tom Woodring,randt901,Tony Randazzo,ortir901,Roberto Ortiz,NaN,NaN,NaN,NaN,counc001,Craig Counsell,weisw001,Walt Weiss,thort001,Tyler Thornburg,rusic001,Chris Rusin,knebc001,Corey Knebel,susaa001,Andrew Susac,cravt001,Tyler Cravy,marqg001,German Marquez,villj001,Jonathan Villar,5.0,genns001,Scooter Gennett,4.0,cartc002,Chris Carter,3.0,santd002,Domingo Santana,9.0,pereh001,Hernan Perez,8.0,arcio002,Orlando Arcia,6.0,susaa001,Andrew Susac,2.0,elmoj001,Jake Elmore,7.0,cravt001,Tyler Cravy,1.0,blacc001,Charlie Blackmon,8.0,dahld001,David Dahl,7.0,arenn001,Nolan Arenado,5.0,gonzc001,Carlos Gonzalez,9.0,murpt002,Tom Murphy,2.0,pattj005,Jordan Patterson,3.0,valap001,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,5,51.0,D,NaN,NaN,NaN,PHI13,36935.0,159.0,000001100,00100031x,33.0,8.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,9.0,1.0,1.0,1.0,0.0,6.0,6.0,3.0,3.0,0.0,0.0,24.0,12.0,3.0,1.0,2.0,0.0,33.0,10.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,2.0,0.0,7.0,5.0,2.0,2.0,0.0,0.0,27.0,7.0,0.0,0.0,1.0,0.0,barkl901,Lance Barksdale,herna901,Angel Hernandez,barrt901,Ted Barrett,littw901,Will Little,NaN,NaN,NaN,NaN,collt801,Terr

Here the meaning of the above column's table:

 * `date` Date in the form "yyyymmdd"
 * `number_of_game`
   * `0` a single game
   * `1` the first game of a double (or triple) header including seperate admission doubleheaders
   * `2` the second game of a double (or triple) header including seperate admission doubleheaders
   * `3` the third game of a triple-header
   * `A` the first game of a double-header involving 3 teams
   * `B` the second game of a double-header involving 3 teams
 * `day_of_week` Day of week  ("Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat")
 * `v_name`, `v_league` Visiting team and league
 * `v_game_number` Visiting team game number. For this and the home team game number, ties are counted as games and suspended games are counted from the starting rather than the ending date.
 * `h_name`, `h_league` Home team and league
 * `h_game_number` Home team game number
 * `v_score`, `h_score` Visiting and home team score (unquoted) 
 * `length_outs` Length of game in outs (unquoted). A full 9-inning game would have a 54 in this field.  If the home team won without batting in the bottom of the ninth, this field would contain a 51.
 * `day_night` Day/night indicator ("D" or "N")
 * `completion` Completion information. If the game was completed at a later date (either due to a suspension or an upheld protest) this field will include `yyyymmdd`, `park`, `vs`, `hs`, `len` where:
   * `yyyymmdd` the date the game was completed
   * `park` the park ID where the game was completed
   * `vs` the visitor score at the time of interruption
   * `hs` the home score at the time of interruption
   * `len` the length of the game in outs at time of interruption
   All the rest of the information in the record refers to the entire game.
 * `forefeit` Forfeit information:
   * `V` the game was forfeited to the visiting team
   * `H` the game was forfeited to the home team
   * `T` the game was ruled a no-decision
 * `protest` Protest information:
   * `P` the game was protested by an unidentified team
   * `V` a disallowed protest was made by the visiting team
   * `H` a disallowed protest was made by the home team
   * `X` an upheld protest was made by the visiting team
   * `Y` an upheld protest was made by the home team
   Note: two of these last four codes can appear in the field (if both teams protested the game).
 * `park_id` Park ID
 * `attendance` Attendance (unquoted)
 * `length_minutes` Time of game in minutes (unquoted)
 * `v_line_score`, `h_line_score` Visiting and home line scores. For example: `010000(10)0x` would indicate a game where the home team scored a run in the second inning, ten in the seventh and didn't bat in the bottom of the ninth.

**Visiting team offensive statistics**

 * `v_at_bats`
 * `v_hits`
 * `v_doubles`
 * `v_triples`
 * `v_homeruns`
 * `v_rbi`
 * `v_sacrifice_hits`
 * `v_sacrifice_flies` may include sacrifice flies for years prior to 1954 when sacrifice flies were allowed.
 * `v_hit_by_pitch`
 * `v_walks`
 * `v_intentional_walks`
 * `v_strikeouts`
 * `v_stolen_bases`
 * `v_caught_stealing`
 * `v_grounded_into_double`
 * `v_first_catcher_interference`
 * `v_left_on_base`
 
**Visiting team pitching statistics**

 * `v_pitchers_used` 1 means it was a complete game
 * `v_individual_earned_runs`
 * `v_team_earned_runs`
 * `v_wild_pitches`
 * `v_balks`
 
**Visiting team defensive statistics**

 * `v_putouts` prior to 1931, this may not equal 3 times the number of innings pitched.  Prior to that, no putout was awarded when a runner was declared out for being hit by a batted ball.
 * `v_assists`
 * `v_errors`
 * `v_passed_balls`
 * `v_double_plays`
 * `v_triple_plays` 
 
**Home team offensive statistics**

 * `h_at_bats`
 * `h_hits`
 * `h_doubles`
 * `h_triples`
 * `h_homeruns`
 * `h_rbi`
 * `h_sacrifice_hits`
 * `h_sacrifice_flies` may include sacrifice flies for years prior to 1954 when sacrifice flies were allowed.
 * `h_hit_by_pitch`
 * `h_walks`
 * `h_intentional_walks`
 * `h_strikeouts`
 * `h_stolen_bases`
 * `h_caught_stealing`
 * `h_grounded_into_double`
 * `h_first_catcher_interference`
 * `h_left_on_base`
 
**Home team pitching statistics**

 * `h_pitchers_used` 1 means it was a complete game
 * `h_individual_earned_runs`
 * `h_team_earned_runs`
 * `h_wild_pitches`
 * `h_balks`
 
**Home team defensive statistics**

 * `h_putouts` prior to 1931, this may not equal 3 times the number of innings pitched.  Prior to that, no putout was awarded when a runner was declared out for being hit by a batted ball.
 * `h_assists`
 * `h_errors`
 * `h_passed_balls`
 * `h_double_plays`
 * `h_triple_plays` 
 
**1B umpire ID and name**

 * `1b_umpire_id` 
 * `1b_umpire_name` 

**2B umpire ID and name**

 * `2b_umpire_id` 
 * `2b_umpire_name` 

**3B umpire ID and name**

 * `3b_umpire_id` 
 * `3b_umpire_name` 

**LF umpire ID and name**

 * `lf_umpire_id` 
 * `lf_umpire_name` 

**RF umpire ID and name**

 * `rf_umpire_id` 
 * `rf_umpire_name` 

(Note: if any umpire positions were not filled for a particular game the fields will be "","(none)".)

**Visiting team manager ID and name**

 * `v_manager_id`
 * `v_manager_name`

**Home team manager ID and name**

 * `h_manager_id`
 * `h_manager_name`

**Winning pitcher ID and name**

 * `winning_pitcher_id`
 * `winning_pitcher_name`

**Losing pitcher ID and name**

 * `losing_pitcher_id`
 * `losing_pitcher_name`

**Saving pitcher ID and name** (Note: "","(none)" if none awarded)

 * `saving_pitcher_id`
 * `saving_pitcher_name`

**Game Winning RBI batter ID and name** (Note: "","(none)" if none awarded)

 * `winning_rbi_batter_id`
 * `winning_rbi_batter_id_name`

**Visiting starting pitcher ID and name**

 * `v_starting_pitcher_id`
 * `v_starting_pitcher_name`

**Home starting pitcher ID and name**

 * `h_starting_pitcher_id`
 * `h_starting_pitcher_name`
 
**Visiting starting players ID, name and defensive position, listed in the order (1-9) they appeared in the batting order.**


 * `v_player_1_id`
 * `v_player_1_name`
 * `v_player_1_def_pos`
 * `v_player_2_id`
 * `v_player_2_name`
 * `v_player_2_def_pos`
 * `v_player_3_id`
 * `v_player_3_name`
 * `v_player_3_def_pos`
 * `v_player_4_id`
 * `v_player_4_name`
 * `v_player_4_def_pos`
 * `v_player_5_id`
 * `v_player_5_name`
 * `v_player_5_def_pos`
 * `v_player_6_id`
 * `v_player_6_name`
 * `v_player_6_def_pos`
 * `v_player_7_id`
 * `v_player_7_name`
 * `v_player_7_def_pos`
 * `v_player_8_id`
 * `v_player_8_name`
 * `v_player_8_def_pos`
 * `v_player_9_id`
 * `v_player_9_name`
 * `v_player_9_def_pos`
 
 **Home starting players ID, name and defensive position listed in the order (1-9) they appeared in the batting order.**
 
 
 * `h_player_1_id`
 * `h_player_1_name`
 * `h_player_1_def_pos`
 * `h_player_2_id`
 * `h_player_2_name`
 * `h_player_2_def_pos`
 * `h_player_3_id`
 * `h_player_3_name`
 * `h_player_3_def_pos`
 * `h_player_4_id`
 * `h_player_4_name`
 * `h_player_4_def_pos`
 * `h_player_5_id`
 * `h_player_5_name`
 * `h_player_5_def_pos`
 * `h_player_6_id`
 * `h_player_6_name`
 * `h_player_6_def_pos`
 * `h_player_7_id`
 * `h_player_7_name`
 * `h_player_7_def_pos`
 * `h_player_8_id`
 * `h_player_8_name`
 * `h_player_8_def_pos`
 * `h_player_9_id`
 * `h_player_9_name`
 * `h_player_9_def_pos`
 
 
 * `additional_info` Additional information. This is a grab-bag of informational items that might not warrant a field on their own. The field is alpha-numeric. Some items are represented by tokens such as `HTBF` (home team batted first) (Note: if "HTBF" is specified it would be possible to see something like "01002000x" in the visitor's line score). Changes in umpire positions during a game will also appear in this field. These will be in the form: `umpchange`, `inning`, `umpPosition`, `umpid` with the latter three repeated for each umpire. These changes occur with umpire injuries, late arrival of umpires or changes from completion of suspended games. Details of suspended games are in field 14.
 * `acquisition_info`
   * `Y` we have the complete game
   * `N` we don't have any portion of the game
   * `D` the game was derived from box score and game story
   * `P` we have some portion of the game. We may be missing innings at the beginning, middle and end of the game.

### <a id='park_codes'></a> park_codes
[Top](#Index)

Information about the park where the match in `game_log` were played (reference `park_id`)

In [7]:
park_codes = data['park_codes']
park_codes.head()

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [8]:
park_codes.tail()

,park_id,name,aka,city,state,start,end,league,notes
247,WIL02,BB&T Ballpark at Bowman Field,NaN,Wiliamsport,PA,08/20/2017,08/20/2017,NL,PIT
248,WNY01,West New York Field Club Grounds,NaN,West New York,NJ,09/11/1898,09/17/1899,NL,"BRO:9/18&10/2/1898; NY1:9/11/98, 6/4&7/16&8/13&9/17/99"
249,WOR01,Agricultural County Fair Grounds I,NaN,Worcester,MA,05/01/1880,09/29/1882,NL,NaN
250,WOR02,Agricultural County Fair Grounds II,NaN,Worcester,MA,08/17/1887,08/17/1887,NL,1 BSN game
251,WOR03,Worcester Driving Park Grounds,NaN,Worcester,MA,10/30/1874,10/30/1874,NaN,1 BS1 game


### <a id='person_codes'></a> person_codes
[Top](#Index)

Information about the players, the references in `game_log` are the columns that includes `id`

In [9]:
person_codes = data['person_codes']
person_codes.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


In [10]:
person_codes.tail()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
20489,zuvep001,Zuvella,Paul,09/04/1982,NaN,04/02/1996,NaN
20490,zuveg101,Zuverink,George,04/21/1951,NaN,NaN,NaN
20491,zwild101,Zwilling,Dutch,08/14/1910,NaN,04/15/1941,NaN
20492,zycht001,Zych,Tony,09/04/2015,NaN,NaN,NaN
20493,thoma102,Thompson,NaN,NaN,NaN,NaN,NaN


### <a id='team_codes'></a> team_codes
[Top](#Index)

Info about teams, referenced in `v_name` and `h_name`

In [11]:
team_codes = data['team_codes']
team_codes.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


In [12]:
team_codes.tail()

,team_id,league,start,end,city,nickname,franch_id,seq
145,WS8,NL,1886,1889,Washington,Senators,WS8,1
146,WS9,AA,1891,1891,Washington,Senators,WS9,1
147,WSN,NL,1892,1899,Washington,Senators,WS9,2
148,WSU,UA,1884,1884,Washington,Nationals,WSU,1
149,MIA,NL,2012,0,Miami,Marlins,FLO,2


### <a id='appearance_type'></a> appearance_type
[Top](#Index)

In [13]:
appearance_type = data['appearance_type']
appearance_type

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


---

## <a href='Additional-info'></a> Additional info
[Top](#Index)

### <a id='Position-of-defensive-players'></a> Position of defensive players
[Top](#Index)

![Alt text](Baseball_positions.svg)

 * `1` **pitcher** - the player who throws the baseball from the pitcher's mound toward the catcher to begin each play, with the goal of retiring a batter, who attempts to either make contact with the pitched ball or draw a walk
 * `2` **catcher** - receives the ball from the pitcher, due to his best position he can direct and lead the other players in a defensive play
 * `3` **first baseman** - is the player on the team playing defense who fields the area nearest first base, and is responsible for the majority of plays made at that base
 * `4` **second baseman** - is a fielding position in the infield, between second and first base
 * `5` **third baseman** - is the player in baseball whose responsibility is to defend the area nearest to third base
 * `6` **shortstop** - is the fielding position between second and third base, which is considered to be among the most demanding defensive positions
 * `7` **left fielder** - also, less commonly known as **left outfield**, is an outfielder who plays defense in left field. Left field is the area of the outfield to the left of a person standing at home plate and facing towards the pitcher's mound
 * `8` **center fielder** - is the outfielder in baseball who plays defense in center field – the fielding position between left field and right field
 * `9` **right fielder** -  is the outfielder who plays defense in right field. Right field is the area of the outfield to the right of a person standing at home plate and facing towards the pitcher's mound
 
An **outfielder** is a person playing in one of the three defensive positions in baseball or softball, farthest from the batter.

### <a id='Leagues-names-and-abbreviations'></a> Leagues' names and abbreviations
[Top](#Index)

 * `NL` - [National League](https://en.wikipedia.org/wiki/National_League)
 * `AL` - [American League](https://en.wikipedia.org/wiki/American_League)
 * `FL` - [Federal League](https://en.wikipedia.org/wiki/Federal_League)
 * `PL` - [Players' League](https://en.wikipedia.org/wiki/Players%27_League)
 * `UA` - [Union Association](https://en.wikipedia.org/wiki/Union_Association)
 * `AA` - [American Association](https://en.wikipedia.org/wiki/American_Association_(19th_century))

### <a id='Offensive-statistics-meaning'></a> Offensive statistics meaning
[Top](#Index)

 * `At bats` (AB) - Plate appearances, not including bases on balls, being hit by pitch, sacrifices, interference, or obstruction.
 * `Hits` (H) - times reached base because of a batted, fair ball without error by the defense
 * `Double` (2B) - hits on which the batter reaches second base safely without the contribution of a fielding error.
 * `Triple` (3B) - hits on which the batter reaches third base safely without the contribution of a fielding error.
 * `Homeruns` (HR) - hits on which the batter successfully touched all four bases, without the contribution of a fielding error.
 * `Run butted in` (RBI) - number of runners who score due to a batters' action, except when batter grounded into double play or reached on an error
 * `Sacrifice hits` (SH) - number of sacrifice bunts which allow runners to advance on the basepaths
 * `Sacrifice Flies` (SF) - fly balls hit to the outfield which although caught for an out, allow a baserunner to advance
 * `Hit by pitch` (HBP) - times touched by a pitch and awarded first base as a result
 * `Walks` (W) - hitter not swinging at four pitches called out of the strike zone and awarded first base (also called Base on balls BB)
 * `Intentional walks` (IW) - times awarded first base on balls (see BB above) deliberately thrown by the pitcher. Also known as Intentional base on balls IBB
 * `Strikeouts` (K) - (also abbreviated SO) number of times that a third strike is taken or swung at and missed, or bunted foul. Catcher must catch the third strike or batter may attempt to run to first base.
 * `Stolen bases` (SB) - number of bases advanced by the runner while the ball is in the possession of the defense
 * `Caught_stealing` (CS) - times tagged out while attempting to steal a base
 * `Grounded into double` (GIDP) - number of ground balls hit that became double plays
 * `First catcher interference` (CI) - Catcher's Interference (e.g., catcher makes contact with bat)
 * `Left on base` (LOB) - baserunners a pitcher does not allow to score

### <a id='Pitching-statistics-meaning'></a> Pitching statistics meaning
[Top](#Index)

 * `Individual earned runs`, `Team earned runs` (ER) - number of runs that did not occur as a result of errors or passed balls 
 * `Wild pitches` (WP) - charged when a pitch is too high, low, or wide of home plate for the catcher to field, thereby allowing one or more runners to advance or score
 * `Balks` (BK) - number of times pitcher commits an illegal pitching action while in contact with the pitching rubber as judged by umpire, resulting in baserunners advancing one base

### <a id='Defensive-statistics'></a> Defensive statistics
[Top](#Index)

 * `Putouts` (PO) - number of times the fielder tags, forces, or appeals a runner and he is called out as a result
 * `Assists` (A) - number of outs recorded on a play where a fielder touched the ball, except if such touching is the putout
 * `Errors` (E) - number of times a fielder fails to make a play he should have made with common effort, and the offense benefits as a result
 * `Passed balls` (PB) - charged to the catcher when the ball is dropped and one or more runners advance
 * `Double_plays` (DP) - one for each double play during which the fielder recorded a putout or an assist
 * `Triple_plays` (TP) - one for each triple play during which the fielder recorded a putout or an assist

---

## <a id='Importing-Data-into-SQLite'></a> Importing Data into SQLite
[Top](#Index)

Exploring the Retrosheet site, we can find this [data dictionary](#http://www.retrosheet.org/eventfile.htm) for their event files, which list every event within each game. This includes the following description:

*id: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, `ATL198304080` should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file.*

You might notice that this essentially makes a custom key using the three columns we identified in our composite key example earlier. After we import the data, we'll construct this column to use as a primary key in our final database.

In [14]:
import_table(game_log, 'game_log');
import_table(park_codes, 'park_codes');
import_table(person_codes, 'person_codes');
import_table(team_codes, 'team_codes');
import_table(appearance_type, 'appearance_type');

In [15]:
run_command('''
ALTER TABLE game_log
ADD COLUMN game_id TEXT
''')

In [16]:
run_command('''
UPDATE game_log
SET game_id = h_name||date||number_of_game
WHERE game_id IS NULL; 
''')

In [17]:
q = """
SELECT
    game_id,
    date,
    h_name,
    number_of_game
FROM game_log
LIMIT 5;
"""

run_query(q)

,game_id,date,h_name,number_of_game
0,FW1187105040,18710504,FW1,0
1,WS3187105050,18710505,WS3,0
2,RC1187105060,18710506,RC1,0
3,CH1187105080,18710508,CH1,0
4,TRO187105090,18710509,TRO,0


---

## <a id='Planning-a-Normalized-Schema'></a> Planning a Normalized Schema
[Top](#Index)

Here the schema of the database

![Alt text](db.png)

`person_appearence` containts id of person in a certain game with a certain role, considering that not all persons are team player the association person-team is in the `team_appearence` table

---

## <a id='Creating-Tables-Without-Foreign-Key-Relations'></a> Creating Tables
[Top](#Index)

### <a id='game'></a> Game
[Top](#Index)

In [18]:
from tables import city

city.drop_table()
city.create_table()
city.feed_table()
city.show_table(5)

(85, 3)


,city_id,name,state
0,1,Albany,NY
1,2,Altoona,PA
2,3,Anaheim,CA
3,4,Arlington,TX
4,5,Atlanta,GA


In [19]:
from tables import league

league.drop_table()
league.create_table()
league.feed_table()
league.show_table()

(6, 2)


,league_id,name
0,NL,National League
1,AL,American League
2,FL,Federal League
3,PL,Players' League
4,UA,Union Association
5,AA,American Association


In [20]:
from tables import park

park.drop_table()
park.create_table()
park.feed_table()
park.show_table(5)

(252, 6)


,park_id,city_id,league_id,name,aka,notes
0,ALB01,1,NL,Riverside Park,None,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,2,UA,Columbia Park,None,None
2,ANA01,3,AL,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,None
3,ARL01,4,AL,Arlington Stadium,None,None
4,ARL02,4,AL,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,None


In [21]:
from tables import game

game.drop_table()
game.create_table()
game.feed_table()
game.show_table(5)

(171907, 13)


,game_id,park_id,date,number_of_game,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,FOR01,18710504,0,54.0,1.0,None,None,None,200.0,120.0,None,Y
1,WS3187105050,WAS01,18710505,0,54.0,1.0,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105060,RCK01,18710506,0,54.0,1.0,None,None,None,1000.0,140.0,None,Y
3,CH1187105080,CHI01,18710508,0,54.0,1.0,None,None,None,5000.0,150.0,None,Y
4,TRO187105090,TRO01,18710509,0,54.0,1.0,None,None,None,3250.0,145.0,HTBF,Y


In [22]:
from tables import person

person.drop_table()
person.create_table()
person.feed_table()
person.show_table(5)

(20494, 3)


,person_id,first,last
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [23]:
from tables import team

team.drop_table()
team.create_table()
team.feed_table()
team.show_table(5)

(149, 6)


,team_id,league_id,city_id,nickname,french_id,seq
0,ALT,UA,2.0,Mountain Cities,ALT,1
1,ARI,NL,NaN,Diamondbacks,ARI,1
2,BFN,NL,10.0,Bisons,BFN,1
3,BFP,PL,10.0,Bisons,BFP,1
4,BL1,None,6.0,Canaries,BL1,1


In [24]:
from tables import person_appearance

person_appearance.drop_table()
person_appearance.create_table()
person_appearance.feed_table()
person_appearance.show_table(5)

(6716280, 4)


,appearance_id,game_id,person_id,appearance_type_id
0,1,ALT188404300,curte801,MM
1,2,ALT188404300,hodnc101,PSP
2,3,ALT188404300,maplb901,UHP
3,4,ALT188404300,murpj104,PSP
4,5,ALT188404300,sullt101,MM


In [25]:
from tables import team_appearence

team_appearence.drop_table()
team_appearence.create_table()
team_appearence.feed_table()
team_appearence.show_table(5)

(5757702, 2)


,appearence_id,team_id
0,1,ALT
1,2,SLU
2,4,ALT
3,5,SLU
4,6,ALT


In [26]:
from tables import team_stat

team_stat.drop_table()
team_stat.create_table()
team_stat.feed_table()
team_stat.show_table(5)

(171907, 33)


,game_id,team_id,game_number,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,FW1187105040,CL1,1,0,000000000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0
1,WS3187105050,BS1,1,20,107000435,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0
2,RC1187105060,CL1,2,12,610020003,49.0,11.0,1.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,NaN,-1.0,NaN,10.0,1.0,0.0,0.0,2.0,0.0,27.0,12.0,8.0,5.0,0.0,0.0
3,CH1187105080,CH1,1,14,077000000,43.0,11.0,2.0,0.0,0.0,8.0,0.0,0.0,0.0,4.0,NaN,2.0,1.0,NaN,-1.0,NaN,6.0,1.0,4.0,4.0,0.0,0.0,27.0,14.0,7.0,2.0,0.0,0.0
4,TRO187105090,BS1,2,9,000002232,46.0,17.0,4.0,1.0,0.0,6.0,0.0,0.0,0.0,2.0,NaN,0.0,1.0,NaN,-1.0,NaN,12.0,1.0,2.0,2.0,0.0,0.0,27.0,12.0,5.0,0.0,1.0,0.0


## <a id='Drop-old-tables'></a> Drop old tables
[Back](#Index)

In [27]:
run_command('DROP TABLE game_log');
run_command('DROP TABLE park_codes');
run_command('DROP TABLE person_codes');
run_command('DROP TABLE team_codes');
run_command('DROP TABLE appearance_type');